In [1]:
import pandas as pd

In [2]:
df10 = pd.read_csv("Ref10_LMI.csv", index_col = 0, dtype = {'SHP_BOROUGH': object, 'SHP_CENSUS_TRACT': object, 'SHP_CENSUS_BLOCK': object, 'Geo_COUNTY': object} )
df11 = pd.read_csv("Ref11_LMI.csv", index_col = 0, dtype = {'SHP_BOROUGH': object, 'SHP_CENSUS_TRACT': object, 'SHP_CENSUS_BLOCK': object, 'Geo_COUNTY': object} )
df12 = pd.read_csv("Ref12_LMI.csv", index_col = 0, dtype = {'SHP_BOROUGH': object, 'SHP_CENSUS_TRACT': object, 'SHP_CENSUS_BLOCK': object, 'Geo_COUNTY': object} )
df13 = pd.read_csv("Ref13_LMI.csv", index_col = 0, dtype = {'SHP_BOROUGH': object, 'SHP_CENSUS_TRACT': object, 'SHP_CENSUS_BLOCK': object, 'Geo_COUNTY': object} )
df14 = pd.read_csv("Ref14_LMI.csv", index_col = 0, dtype = {'SHP_BOROUGH': object, 'SHP_CENSUS_TRACT': object, 'SHP_CENSUS_BLOCK': object, 'Geo_COUNTY': object} )

In [3]:
df10['count'] = 1
df11['count'] = 1 
df12['count'] = 1 
df13['count'] = 1 
df14['count'] = 1

In [4]:
df10_grouped = df10.groupby(['PRMSICD', 'PNACODE', 'PNATITL', 'Geo_COUNTY', 'SHP_BOROUGH', 'SHP_CENSUS_TRACT', 'lmi_ct'])['EMPSDT', 'SLSVDT', 'count'].agg(['sum', 'median']).reset_index()
df11_grouped = df11.groupby(['SICD', 'PNACODE', 'PNATITL', 'Geo_COUNTY', 'SHP_BOROUGH', 'SHP_CENSUS_TRACT', 'lmi_ct'])['EMPSDT', 'SLSVDT', 'count'].agg(['sum', 'median']).reset_index()
df12_grouped = df12.groupby(['SICD', 'PNACODE', 'PNATITL', 'Geo_COUNTY', 'SHP_BOROUGH', 'SHP_CENSUS_TRACT', 'lmi_ct'])['EMPSDT', 'SLSVDT', 'count'].agg(['sum', 'median']).reset_index()
df13_grouped = df13.groupby(['SICD', 'PNACODE', 'PNATITL', 'Geo_COUNTY', 'SHP_BOROUGH', 'SHP_CENSUS_TRACT', 'lmi_ct'])['EMPSDT', 'SLSVDT', 'count'].agg(['sum', 'median']).reset_index()
df14_grouped = df14.groupby(['SICD', 'PNACODE', 'PNATITL', 'Geo_COUNTY', 'SHP_BOROUGH', 'SHP_CENSUS_TRACT', 'lmi_ct'])['EMPSDT', 'SLSVDT', 'count'].agg(['sum', 'median']).reset_index()

In [5]:
cols = ['SICD', 'PNACODE', 'PNATITL', 'Geo_COUNTY', 'SHP_BOROUGH', 'SHP_CENSUS_TRACT', 'lmi_ct', \
        'EMPSDT_SUM', 'EMPSDT_MED', 'SLSVDT_SUM', 'SLSVDT_MED', 'count_SUM', 'count_MED']

In [6]:
df10_grouped.columns = df10_grouped.columns.droplevel(1)
df11_grouped.columns = df11_grouped.columns.droplevel(1)
df12_grouped.columns = df12_grouped.columns.droplevel(1)
df13_grouped.columns = df13_grouped.columns.droplevel(1)
df14_grouped.columns = df14_grouped.columns.droplevel(1)

In [7]:
df10_grouped.columns = cols
df11_grouped.columns = cols
df12_grouped.columns = cols
df13_grouped.columns = cols
df14_grouped.columns = cols

In [8]:
del df10_grouped['count_MED']
del df11_grouped['count_MED']
del df12_grouped['count_MED']
del df13_grouped['count_MED']
del df14_grouped['count_MED']

####don't use below code - no need to save

In [ ]:
df10_grouped.to_csv("df10_grouped.csv")
df11_grouped.to_csv("df11_grouped.csv")
df12_grouped.to_csv("df12_grouped.csv") 
df13_grouped.to_csv("df13_grouped.csv")
df14_grouped.to_csv("df14_grouped.csv") 

In [9]:
import geopandas as gp

In [10]:
boros = gp.GeoDataFrame.from_file("../nyct2010_15b/nyct2010.shp")

In [120]:
result = pd.merge(df10_grouped, boros[['BoroName', 'CT2010', 'NTAName', 'Shape_Area', 'geometry']], \
                   left_on = ['SHP_BOROUGH', 'SHP_CENSUS_TRACT'], right_on = ['BoroName', 'CT2010'])


In [42]:
#gp.GeoDataFrame(result[result['PNATITL'] == 'OTHER SOCIAL ADVOCACY ORGANIZATIONS']).to_json()

In [44]:
# test = gp.GeoDataFrame.from_file('sweetgreen.geojson')
# test.to_file("test")

####test drive

In [107]:
df = df10_grouped[df10_grouped['SICD'] == 'BEAUTY SALONS']

In [109]:
result = pd.merge(df, boros[['BoroName', 'CT2010', 'NTAName', 'Shape_Area', 'geometry']], \
                  left_on = ['SHP_BOROUGH', 'SHP_CENSUS_TRACT'], right_on = ['BoroName', 'CT2010'])
del result['BoroName']
del result['CT2010']

In [142]:
cnt = len(PNATITL)
base_loc = ''
base_year_loc = base_loc + "/" + '2010'
year_dir = os.listdir(base_year_loc)

def createGeoJSON(pnatitl_loc, string2):
    
    
    #pnatitl_loc = base_year_loc + "/" + string1
    sicd_code = string2.replace("/", "-") 
    
    df = df10_grouped[df10_grouped['SICD'] == string2]
    
    result = pd.merge(df, boros[['BoroName', 'CT2010', 'NTAName', 'Shape_Area', 'geometry']], \
                  left_on = ['SHP_BOROUGH', 'SHP_CENSUS_TRACT'], right_on = ['BoroName', 'CT2010'])
    
    loc = pnatitl_loc + "/" + sicd_code + ".json"
    
    with open(loc, 'wb') as fp:
        json.dump(gp.GeoDataFrame(result).to_json(), fp)

#gp.GeoDataFrame(result).to_json()

In [121]:
import json

In [122]:
createGeoJSON("AC REFRIGERATION & FORCED AIR HEATING", "AIR CONDITIONING ROOM UNITS-MFRS")

####create directories 

In [55]:
import os

In [56]:
mkdir geojson

In [48]:
YEARS = [2010, 2011, 2012, 2013, 2014]
PNATITL = list(df10_grouped.groupby(['PNATITL', 'SICD']).agg('count').index.get_level_values(0))
SICD = list(df10_grouped.groupby(['PNATITL', 'SICD']).agg('count').index.get_level_values(1))

In [61]:
for year in YEARS:
    year_folder = str(year)
    
    if year_folder in os.listdir(base_dir):
        print 'already created'
    else:
        add_year = base_dir + "/" + year_folder
        os.mkdir(add_year)

already created
already created
already created
already created
already created


In [92]:
apm = base_year_loc + "/" + "ABRASIVE PROD MANUFACTURING"
ac = base_year_loc + "/"  + "AC REFRIGERATION & FORCED AIR HEATING"

os.mkdir(apm)
os.mkdir(ac)

In [146]:
cnt = len(PNATITL)
base_loc = 
base_year_loc = base_loc + "/" + '2010'
year_dir = os.listdir(base_year_loc)

for index in range(cnt): 
    
    year_dir = os.listdir(base_year_loc)
    
    pnatitl = PNATITL[index].replace("/", "-")
    sicd = SICD[index]
    
    if pnatitl not in year_dir: 
        
        pnatitl_loc = base_year_loc + "/" + pnatitl
        os.mkdir(pnatitl_loc)
          
        createGeoJSON(pnatitl_loc, sicd)

    else:
        pnatitl_loc = base_year_loc + "/" + pnatitl
        createGeoJSON(pnatitl_loc, sicd)

In [136]:
# df10_grouped.groupby(['PNATITL', 'SICD']).agg('count').head(10)

x = "INTERNET PUBLISHING/BROADCASTING/WEB SEARCH PRTLS"

In [138]:
x.replace("/", "-")

'INTERNET PUBLISHING-BROADCASTING-WEB SEARCH PRTLS'

In [172]:
result = df10_grouped[df10_grouped['PNATITL'] == 'DIRECT MAIL ADVERTISING']


In [174]:
result1 = pd.merge(result, boros[['BoroName', 'CT2010', 'NTAName', 'Shape_Area', 'geometry']], \
                  left_on = ['SHP_BOROUGH', 'SHP_CENSUS_TRACT'], right_on = ['BoroName', 'CT2010'])

In [182]:
#gp.GeoDataFrame(result1).to_json()
#result['SHP_CENSUS_TRACT'].value_counts()

In [ ]:
f